# COMPARE DIFFERENT IMPLEMENTATIONS

### IMPORTS

Perform imports and check environment.

In [1]:
import pandas as pd
import torch
from multiprocessing import Pool, cpu_count
import numpy as np
from src.input.write_files import write_int_matrix_file
from src.input.read_files import find_files, numbers_from_file_name, normal_sec_size_int_from_file_name, number_from_text
from datetime import datetime
from src.algorithm.vanilla.circuit_synthesis import synthesise_circuit as synthesise_circuit_vanilla
from src.algorithm.torch.circuit_synthesis import synthesise_circuit as synthesise_circuit_torch
from src.algorithm.vanilla.execution import load_matrix_and_benchmark
from src.algorithm.torch.execution import load_tensor_and_benchmark
import csv

print(f'There are {cpu_count()} threads available on this machine.')
print(f'CUDA is{" " if torch.cuda.is_available() else " not "}available on this machine.')

There are 12 threads available on this machine.
CUDA is available on this machine.


### DEFINING PARAMETERS

Defining the parameters for the input generation and the shared parameters for the benchmarks.

In [2]:
sample_size = 2
lower_size = 10
upper_size = 100
step_size = 5

sizes = np.arange(lower_size, upper_size + step_size, step_size)
samples = np.arange(1, sample_size + 1, 1)

input_gen_processors = 10

input_dir = r'resources\comparison\input_matrices'
input_file_prefix = 'SingMat'

output_dir = r'resources\comparison'
output_file_prefix = 'AlgorithmComparison'

output_file_fieldnames = ['size', 'run', 'sec_size', 'num_gates', 'process_time', 'initial_rss', 'final_rss']

### INPUT GENERATION

Generate the matrices and save them to files.

In [3]:
input_gen_argument_tuples = ([(size, size * size, rf'..\{input_dir}\{input_file_prefix}_{size}_{sample}.txt') for size in sizes for sample in samples])

[print(argument_tuple) for argument_tuple in input_gen_argument_tuples[:min(10, len(input_gen_argument_tuples))]]

(10, 100, '..\\resources\\comparison\\input_matrices\\SingMat_10_1.txt')
(10, 100, '..\\resources\\comparison\\input_matrices\\SingMat_10_2.txt')
(15, 225, '..\\resources\\comparison\\input_matrices\\SingMat_15_1.txt')
(15, 225, '..\\resources\\comparison\\input_matrices\\SingMat_15_2.txt')
(20, 400, '..\\resources\\comparison\\input_matrices\\SingMat_20_1.txt')
(20, 400, '..\\resources\\comparison\\input_matrices\\SingMat_20_2.txt')
(25, 625, '..\\resources\\comparison\\input_matrices\\SingMat_25_1.txt')
(25, 625, '..\\resources\\comparison\\input_matrices\\SingMat_25_2.txt')
(30, 900, '..\\resources\\comparison\\input_matrices\\SingMat_30_1.txt')
(30, 900, '..\\resources\\comparison\\input_matrices\\SingMat_30_2.txt')


[None, None, None, None, None, None, None, None, None, None]

In [4]:
print(f'Generating {len(input_gen_argument_tuples)} matrices in size range from {lower_size} to {upper_size} ...')

if __name__ ==  '__main__':
    with Pool(processes = input_gen_processors) as pool:
        pool.starmap(write_int_matrix_file, input_gen_argument_tuples)

print(f'Saved matrices to {input_dir} ...')

Generating 38 matrices in size range from 10 to 100 ...
Saved matrices to resources\comparison\input_matrices ...


### GET FILES TO BENCHMARK

Find relevant files and sort them.

In [5]:
file_regex = rf'({input_file_prefix}).*\.txt'
size_regex = r'_\d+_'
sample_regex = r'_\d+\.'

input_file_names = find_files(rf'..\{input_dir}', file_regex)
input_file_names.sort(key = lambda input_file_name: numbers_from_file_name(input_file_name, size_regex, sample_regex))

[print(input_file_name) for input_file_name in input_file_names[:min(10, len(input_file_names))]]

SingMat_10_1.txt
SingMat_10_2.txt
SingMat_15_1.txt
SingMat_15_2.txt
SingMat_20_1.txt
SingMat_20_2.txt
SingMat_25_1.txt
SingMat_25_2.txt
SingMat_30_1.txt
SingMat_30_2.txt


[None, None, None, None, None, None, None, None, None, None]

### GET DIFFERENT SECTION SIZES

Get the normal section sizes and produce input tuples together with the files.

In [6]:
section_sizes_df = pd.read_csv(rf'..\resources\optimization\SecSizes_2_5000.csv')
section_sizes_df = section_sizes_df.loc[(section_sizes_df['size'] >= lower_size) & (section_sizes_df['size'] <= upper_size)][['size', 'norm_sec_size']]
section_sizes_df.head()

,size,norm_sec_size
8,10,2
9,11,2
10,12,2
11,13,2
12,14,2


In [7]:
section_sizes_pivot_df = section_sizes_df.set_index('size').stack().reset_index(name='sec_size').rename(columns={'level_1':'sec_size_type'})
section_sizes_pivot_df.head()

,size,sec_size_type,sec_size
0,10,norm_sec_size,2
1,11,norm_sec_size,2
2,12,norm_sec_size,2
3,13,norm_sec_size,2
4,14,norm_sec_size,2


In [8]:
file_name_arguments = [(number_from_text(input_file_name, size_regex),
                        number_from_text(input_file_name, sample_regex),
                        input_file_name)
                       for input_file_name in input_file_names]
file_name_arguments_df = pd.DataFrame(file_name_arguments, columns =['size', 'sample', 'path'])
file_name_arguments_df.head()

,size,sample,path
0,10,1,SingMat_10_1.txt
1,10,2,SingMat_10_2.txt
2,15,1,SingMat_15_1.txt
3,15,2,SingMat_15_2.txt
4,20,1,SingMat_20_1.txt


In [9]:
benchmark_tuples_df = pd.merge(section_sizes_pivot_df, file_name_arguments_df, on='size', how='inner')[['size', 'sec_size', 'sample', 'path']]
benchmark_tuples_df.head()

,size,sec_size,sample,path
0,10,2,1,SingMat_10_1.txt
1,10,2,2,SingMat_10_2.txt
2,15,2,1,SingMat_15_1.txt
3,15,2,2,SingMat_15_2.txt
4,20,2,1,SingMat_20_1.txt


In [10]:
benchmark_tuples_list = [tuple(benchmark_tuple) for benchmark_tuple in benchmark_tuples_df.values.tolist()]

[print(benchmark_tuple) for benchmark_tuple in benchmark_tuples_list[:min(10, len(benchmark_tuples_list))]]

(10, 2, 1, 'SingMat_10_1.txt')
(10, 2, 2, 'SingMat_10_2.txt')
(15, 2, 1, 'SingMat_15_1.txt')
(15, 2, 2, 'SingMat_15_2.txt')
(20, 2, 1, 'SingMat_20_1.txt')
(20, 2, 2, 'SingMat_20_2.txt')
(25, 2, 1, 'SingMat_25_1.txt')
(25, 2, 2, 'SingMat_25_2.txt')
(30, 2, 1, 'SingMat_30_1.txt')
(30, 2, 2, 'SingMat_30_2.txt')


[None, None, None, None, None, None, None, None, None, None]

### BENCHMARK VANILLA

Prepare the arguments.

Read each file into a matrix, perform a benchmark and save the results in a file.

In [11]:
vanilla_output_file_postfix = 'Vanilla'

vanilla_benchmark_argument_tuples = [(synthesise_circuit_vanilla,
                                    size,
                                    sec_size,
                                    sample,
                                    rf'..\{input_dir}\{input_file_name}')
                                   for (size, sec_size, sample, input_file_name) in benchmark_tuples_list]

In [12]:
output_file_timestamp = datetime.now().strftime('%d%m%y_%H%M')

vanilla_result_tuples = []

for vanilla_benchmark_argument_tuple in vanilla_benchmark_argument_tuples:

    vanilla_result_tuples.append(load_matrix_and_benchmark(*vanilla_benchmark_argument_tuple))

print(f'Saving results to {output_dir} ...')

with open(rf'..\{output_dir}\{output_file_prefix}_{vanilla_output_file_postfix}_{output_file_timestamp}.csv', mode='w', newline='') as output_file:
    writer = csv.DictWriter(output_file, fieldnames=output_file_fieldnames)
    writer.writeheader()

    for (size, run, sec_size, num_gates, process_time, initial_rss, final_rss) in vanilla_result_tuples:

        writer.writerow({'size': size, 'run': run, 'sec_size': sec_size, 'num_gates': num_gates, 'process_time': process_time, 'initial_rss': initial_rss, 'final_rss': final_rss})

print('Done.')

Loading matrix from ..\resources\comparison\input_matrices\SingMat_10_1.txt ...
Benchmarking sample 1 ... matrix size: 10, section size: 2
Loading matrix from ..\resources\comparison\input_matrices\SingMat_10_2.txt ...
Benchmarking sample 2 ... matrix size: 10, section size: 2
Loading matrix from ..\resources\comparison\input_matrices\SingMat_15_1.txt ...
Benchmarking sample 1 ... matrix size: 15, section size: 2
Loading matrix from ..\resources\comparison\input_matrices\SingMat_15_2.txt ...
Benchmarking sample 2 ... matrix size: 15, section size: 2
Loading matrix from ..\resources\comparison\input_matrices\SingMat_20_1.txt ...
Benchmarking sample 1 ... matrix size: 20, section size: 2
Loading matrix from ..\resources\comparison\input_matrices\SingMat_20_2.txt ...
Benchmarking sample 2 ... matrix size: 20, section size: 2
Loading matrix from ..\resources\comparison\input_matrices\SingMat_25_1.txt ...
Benchmarking sample 1 ... matrix size: 25, section size: 2
Loading matrix from ..\reso

### BENCHMARK TORCH ON CUDA

Configure the environment for a PyTorch benchmark on CUDA and prepare the arguments.

Read each file into a tensor, perform a benchmark and save the results in a file.

In [13]:
torch_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'The PyTorch implementation will be executed on the device {torch_device}.')

torch_output_file_postfix = 'TorchCUDA'

torch_benchmark_argument_tuples = [(synthesise_circuit_torch,
                                    size,
                                    sec_size,
                                    sample,
                                    rf'..\{input_dir}\{input_file_name}',
                                    torch_device)
                                   for (size, sec_size, sample, input_file_name) in benchmark_tuples_list]

The PyTorch implementation will be executed on the device cuda.


In [14]:
output_file_timestamp = datetime.now().strftime('%d%m%y_%H%M')

torch_result_tuples = []

for torch_benchmark_argument_tuple in torch_benchmark_argument_tuples:

    torch_result_tuples.append(load_tensor_and_benchmark(*torch_benchmark_argument_tuple))

print(f'Saving results to {output_dir} ...')

with open(rf'..\{output_dir}\{output_file_prefix}_{torch_output_file_postfix}_{output_file_timestamp}.csv', mode='w', newline='') as output_file:
    writer = csv.DictWriter(output_file, fieldnames=output_file_fieldnames)
    writer.writeheader()

    for (size, run, sec_size, num_gates, process_time, initial_rss, final_rss) in torch_result_tuples:

        writer.writerow({'size': size, 'run': run, 'sec_size': sec_size, 'num_gates': num_gates, 'process_time': process_time, 'initial_rss': initial_rss, 'final_rss': final_rss})

print('Done.')


Loading matrix from ..\resources\comparison\input_matrices\SingMat_10_1.txt and allocating tensor ...
Tensor allocated on cuda ...
Benchmarking sample 1 ... matrix size: 10, section size: 2
Loading matrix from ..\resources\comparison\input_matrices\SingMat_10_2.txt and allocating tensor ...
Tensor allocated on cuda ...
Benchmarking sample 2 ... matrix size: 10, section size: 2
Loading matrix from ..\resources\comparison\input_matrices\SingMat_15_1.txt and allocating tensor ...
Tensor allocated on cuda ...
Benchmarking sample 1 ... matrix size: 15, section size: 2
Loading matrix from ..\resources\comparison\input_matrices\SingMat_15_2.txt and allocating tensor ...
Tensor allocated on cuda ...
Benchmarking sample 2 ... matrix size: 15, section size: 2
Loading matrix from ..\resources\comparison\input_matrices\SingMat_20_1.txt and allocating tensor ...
Tensor allocated on cuda ...
Benchmarking sample 1 ... matrix size: 20, section size: 2
Loading matrix from ..\resources\comparison\input_